In [603]:
import pandas as pd
import requests
import time
pd.set_option("display.max_columns", None)

##### Hacemos llamada inicial para saber si funciona y tenemos acceso a la página.
##### Establecemos parametros de la busqueda
##### Cambiamos parametros varias veces para buscar cada género por año, ya que la API solo nos permite estos parametros de búsqueda.

In [604]:


url_endpoint = "https://moviesdatabase.p.rapidapi.com/titles"

querystring = {"genre": "Drama","titleType":"short","year":"2024","page":"1"}  # obtuvimos estos parametros en la pagina API

headers = {
	"X-RapidAPI-Key": "1db777625dmshdce6ed30bc0ec06p198b62jsn978c35c52050",
	"X-RapidAPI-Host": "moviesdatabase.p.rapidapi.com"
}

llamada = requests.get(url_endpoint, headers=headers, params=querystring)

print(llamada.json())

llamada = requests.get(url_endpoint, headers=headers, params=querystring)

print(llamada.json())
resultados_llamada = llamada.json()

{'page': '1', 'next': '/titles?titleType=short&page=2&year=2024&genre=Drama', 'entries': 10, 'results': [{'_id': '61e5a2e4ef99b1c5434975db', 'id': 'tt14063056', 'primaryImage': None, 'titleType': {'displayableProperty': {'value': {'plainText': 'Short', '__typename': 'Markdown'}, '__typename': 'DisplayableTitleTypeProperty'}, 'text': 'Short', 'id': 'short', 'isSeries': False, 'isEpisode': False, 'categories': [{'value': 'movie', '__typename': 'TitleTypeCategory'}], 'canHaveEpisodes': False, '__typename': 'TitleType'}, 'titleText': {'text': 'Before the Fish Fell', '__typename': 'TitleText'}, 'originalTitleText': {'text': 'Before the Fish Fell', '__typename': 'TitleText'}, 'releaseYear': {'year': 2024, 'endYear': None, '__typename': 'YearRange'}, 'releaseDate': {'day': 1, 'month': 3, 'year': 2024, '__typename': 'ReleaseDate'}}, {'_id': '61e5a2f2ef99b1c54349794f', 'id': 'tt14072204', 'primaryImage': None, 'titleType': {'text': 'Short', 'id': 'short', 'isSeries': False, 'isEpisode': False, 

##### Vemos que la llamada.json contiene 5 elementos: 'page' (el numero de la pagina), 'next' (la direccion exacta de la pagina), 'entries' (el numero de peliculas en la pagina) y 'results' que contiene toda la información que necesitamos.
##### Ahora  creamos una funcion para acceder a más paginas (no solo la primera) con bucle while que pase a la siguiente página despues de obtener la información.
##### A base de pueba y error nos hemos dado cuenta que los resultados por género tienen un máximo de 50 páginas, luego vuelve a la página 1 y nos devuelve peliculas repetidas. Así que ponemos un límite 50 en nuestro bucle.

In [605]:
def llamada_paginada(url_endpoint, headers, params):
    pagina = 1
    data_peliculas = []
    while pagina <= 50:   # Aqui limitamos el numero de paginas que vamos a llamar ejemplo: ' while pagina <= 10' llama solo 10 paginas, pero si usamos 'while True' extrae todas las paginas hasta que  'next' no exista.
        params['page'] = pagina
        llamada = requests.get(url_endpoint, headers=headers, params=params)
        
        if llamada.status_code == 200:
            llamada= llamada.json()
            data_peliculas.extend(llamada['results'])  # extraemos los datos de la clave 'resutls'
            print(f"Página {pagina}: {len(llamada['results'])} películas extraídas.")
            pagina += 1
                
        elif llamada.status_code == 429: # 429 es error que devuelve API cuando hemos hecho muchas solicitudes.
            print("Demasiadas solicitudes. Esperando para reintentar...")
            time.sleep(5)  # Espera antes de reintentar.
            continue  # Vuelve a intentar la misma página.
        
        else:
            print(f"Error al buscar datos: {llamada.status_code} {llamada.reason}")
            break
    # Comprobamos si existe otra pagina.
        if not llamada.get('next'): #si no hay mas 'next' en la pagina...
            print("Ultima pagina extraida.")
            break
        time.sleep(2) # Espera entre llamadas a la API para evitar sobrecarga y que nos restringan acceso a la pagina.

    return data_peliculas
            
        

##### Ahora llamamos a la funcion que creamos con la API moviesdatabase.

In [606]:
resultados_paginados = llamada_paginada(url_endpoint, headers=headers, params=querystring)    

Página 1: 10 películas extraídas.
Página 2: 10 películas extraídas.
Página 3: 10 películas extraídas.
Página 4: 10 películas extraídas.
Página 5: 10 películas extraídas.
Página 6: 10 películas extraídas.
Página 7: 10 películas extraídas.
Página 8: 10 películas extraídas.
Página 9: 10 películas extraídas.
Página 10: 10 películas extraídas.
Página 11: 10 películas extraídas.
Página 12: 10 películas extraídas.
Página 13: 10 películas extraídas.
Página 14: 10 películas extraídas.
Página 15: 10 películas extraídas.
Página 16: 10 películas extraídas.
Página 17: 10 películas extraídas.
Página 18: 10 películas extraídas.
Página 19: 10 películas extraídas.
Página 20: 10 películas extraídas.
Página 21: 10 películas extraídas.
Página 22: 10 películas extraídas.
Página 23: 10 películas extraídas.
Página 24: 10 películas extraídas.
Página 25: 10 películas extraídas.
Página 26: 10 películas extraídas.
Página 27: 10 películas extraídas.
Página 28: 10 películas extraídas.
Página 29: 10 películas extra

##### Visualizamos resultados

In [607]:
# type(resultados_paginados)
# resultados_paginados

In [608]:
def organizar_data(resultados_json):
    peli_data = []
    for i in resultados_json:
        # Extraer los datos necesarios
        peli_id = i['id'] # 'id' es donde se encuentran los ids de los elementos en la estructura de la pagina API.
        peli_type = i['titleType']['text'] if 'titleType' in i else 'Desconocido'  # 'tiltType' es donde se encuentran el tipo elemento(ejem: movie, tvshow.) en la estructura de la pagina API.
        peli_nombre = i['titleText']['text'] if 'titleText' in i else 'Desconocido' # 'titleText' es donde se encuentran los titulos de las peliculas en la estructura de la pagina API.
        
        # Manejo seguro de 'releaseDate' para evitar AttributeError en caso que no este disponible el año de estreno.
        release_date = i.get('releaseDate', None)  # Usa None como valor por defecto
        if release_date is None:  # Si releaseDate es None, usar valores por defecto
            peli_año = 'Desconocido'
            peli_mes = 'Desconocido'
        else:
            peli_año = release_date.get('year', 'Desconocido')
            peli_mes = release_date.get('month', 'Desconocido')
        
        # Crear el diccionario con los datos 
        movie_dict = {
            'Tipo': peli_type,
            'Nombre': peli_nombre,
            'Año de Estreno': peli_año,
            'Mes de Estreno': peli_mes,
            'ID de la Película': peli_id
        }
        peli_data.append(movie_dict)
    
    print(peli_data)
    # Convertir la lista de diccionarios en un DataFrame
    return pd.DataFrame(peli_data)



In [609]:
df_pelis = organizar_data(resultados_paginados)

[{'Tipo': 'Short', 'Nombre': 'Before the Fish Fell', 'Año de Estreno': 2024, 'Mes de Estreno': 3, 'ID de la Película': 'tt14063056'}, {'Tipo': 'Short', 'Nombre': 'Eddie Thurber-V-My Darkness Ended on that Halloween...', 'Año de Estreno': 2024, 'Mes de Estreno': 10, 'ID de la Película': 'tt14072204'}, {'Tipo': 'Short', 'Nombre': 'Det Kan Ingen Ta Ifrån Mig', 'Año de Estreno': 2024, 'Mes de Estreno': 12, 'ID de la Película': 'tt15103886'}, {'Tipo': 'Short', 'Nombre': 'The Library', 'Año de Estreno': 2024, 'Mes de Estreno': 1, 'ID de la Película': 'tt17062072'}, {'Tipo': 'Short', 'Nombre': 'Somnus', 'Año de Estreno': 2024, 'Mes de Estreno': 3, 'ID de la Película': 'tt18335394'}, {'Tipo': 'Short', 'Nombre': 'Two Questions and a Wish', 'Año de Estreno': 2024, 'Mes de Estreno': 5, 'ID de la Película': 'tt18388426'}, {'Tipo': 'Short', 'Nombre': 'The LGBTQ+ Films: Q', 'Año de Estreno': 2024, 'Mes de Estreno': 8, 'ID de la Película': 'tt20193284'}, {'Tipo': 'Short', 'Nombre': 'The LGBTQ+ Films:

In [610]:
df_pelis

,Tipo,Nombre,Año de Estreno,Mes de Estreno,ID de la Película
0,Short,Before the Fish Fell,2024,3,tt14063056
1,Short,Eddie Thurber-V-My Darkness Ended on that Hall...,2024,10,tt14072204
2,Short,Det Kan Ingen Ta Ifrån Mig,2024,12,tt15103886
3,Short,The Library,2024,1,tt17062072
4,Short,Somnus,2024,3,tt18335394
...,...,...,...,...,...
495,Short,Steve the Queen,2024,1,tt30818291
496,Short,Morality of Murder Scene - Barry Adaptation,2024,1,tt30818607
497,Short,All the Words But the One,Desconocido,Desconocido,tt30818627
498,Short,HR Lady,2024,3,tt30818686


##### Aqui convertimos los resultados 'limpios' a csv. Lo hacemos con todas las páginas (Acción).

In [611]:
#df_pelis.to_csv("Accion_2010.csv", index=False, encoding='utf-8')
# df_pelis.to_csv("Accion_2011.csv", index=False, encoding='utf-8')
#df_pelis.to_csv("Accion_2012.csv", index=False, encoding='utf-8')
#df_pelis.to_csv("Accion_2013.csv", index=False, encoding='utf-8')
# df_pelis.to_csv("Accion_2014.csv", index=False, encoding='utf-8')
#df_pelis.to_csv("Accion_2015.csv", index=False, encoding='utf-8')
#df_pelis.to_csv("Accion_2017.csv", index=False, encoding='utf-8')
#df_pelis.to_csv("Accion_2018.csv", index=False, encoding='utf-8')
#df_pelis.to_csv("Accion_2019.csv", index=False, encoding='utf-8')
#df_pelis.to_csv("Accion_2020.csv", index=False, encoding='utf-8')
#df_pelis.to_csv("Accion_2021.csv", index=False, encoding='utf-8')
#df_pelis.to_csv("Accion_2022.csv", index=False, encoding='utf-8')
#df_pelis.to_csv("Accion_2023.csv", index=False, encoding='utf-8')
#df_pelis.to_csv("Accion_2024.csv", index=False, encoding='utf-8')


##### Aqui convertimos los resultados 'limpios' a csv. Lo hacemos con todas las páginas (Comedia).

In [612]:
#df_pelis.to_csv("Comedia_2010.csv", index=False, encoding='utf-8')
#df_pelis.to_csv("Comedia_2011.csv", index=False, encoding='utf-8')
#df_pelis.to_csv("Comedia_2012.csv", index=False, encoding='utf-8')
#df_pelis.to_csv("Comedia_2013.csv", index=False, encoding='utf-8')
#df_pelis.to_csv("Comedia_2014.csv", index=False, encoding='utf-8')
#df_pelis.to_csv("Comedia_2015.csv", index=False, encoding='utf-8')
#df_pelis.to_csv("Comedia_2016.csv", index=False, encoding='utf-8')
#df_pelis.to_csv("Comedia_2017.csv", index=False, encoding='utf-8')
#df_pelis.to_csv("Comedia_2018.csv", index=False, encoding='utf-8')
#df_pelis.to_csv("Comedia_2019.csv", index=False, encoding='utf-8')
#df_pelis.to_csv("Comedia_2020.csv", index=False, encoding='utf-8')
# df_pelis.to_csv("Comedia_2021.csv", index=False, encoding='utf-8')
#df_pelis.to_csv("Comedia_2022.csv", index=False, encoding='utf-8')
#df_pelis.to_csv("Comedia_2023.csv", index=False, encoding='utf-8')
#df_pelis.to_csv("Comedia_2024.csv", index=False, encoding='utf-8')
# df_pelis.to_csv("Comedia_2025.csv", index=False, encoding='utf-8')

##### Aqui convertimos los resultados 'limpios' a csv. Lo hacemos con todas las páginas (Drama).

In [613]:
#df_pelis.to_csv("Drama_2010.csv", index=False, encoding='utf-8')
# df_pelis.to_csv("Drama_2011.csv", index=False, encoding='utf-8')
#df_pelis.to_csv("Drama_2012.csv", index=False, encoding='utf-8')
#df_pelis.to_csv("Drama_2013.csv", index=False, encoding='utf-8')
#df_pelis.to_csv("Drama_2014.csv", index=False, encoding='utf-8')
#df_pelis.to_csv("Drama_2015.csv", index=False, encoding='utf-8')
#df_pelis.to_csv("Drama_2016.csv", index=False, encoding='utf-8')
#df_pelis.to_csv("Drama_2017.csv", index=False, encoding='utf-8')
#df_pelis.to_csv("Drama_2018.csv", index=False, encoding='utf-8')
#df_pelis.to_csv("Drama_2019.csv", index=False, encoding='utf-8')
# df_pelis.to_csv("Drama_2020.csv", index=False, encoding='utf-8')
#df_pelis.to_csv("Drama_2021.csv", index=False, encoding='utf-8')
#df_pelis.to_csv("Drama_2022.csv", index=False, encoding='utf-8')
#df_pelis.to_csv("Drama_2023.csv", index=False, encoding='utf-8')
# df_pelis.to_csv("Drama_2024.csv", index=False, encoding='utf-8')


#### Cortometrajes Accion

In [614]:
# df_pelis.to_csv("accion_2010.csv", index=False, encoding='utf-8')
# df_pelis.to_csv("accion_2011.csv", index=False, encoding='utf-8')
#df_pelis.to_csv("accion_2012.csv", index=False, encoding='utf-8')
#df_pelis.to_csv("accion_2013.csv", index=False, encoding='utf-8')
#df_pelis.to_csv("accion_2014.csv", index=False, encoding='utf-8')
#df_pelis.to_csv("accion_2015.csv", index=False, encoding='utf-8')
#df_pelis.to_csv("accion_2016.csv", index=False, encoding='utf-8')
#df_pelis.to_csv("accion_2017.csv", index=False, encoding='utf-8')
#df_pelis.to_csv("accion_2018.csv", index=False, encoding='utf-8')
#df_pelis.to_csv("accion_2019.csv", index=False, encoding='utf-8')
#df_pelis.to_csv("accion_2020.csv", index=False, encoding='utf-8')
#df_pelis.to_csv("accion_2021.csv", index=False, encoding='utf-8')
#df_pelis.to_csv("accion_2022.csv", index=False, encoding='utf-8')
#df_pelis.to_csv("accion_2023.csv", index=False, encoding='utf-8')
#df_pelis.to_csv("accion_2024.csv", index=False, encoding='utf-8')

#### Cortometrajes Comedia

In [615]:
#df_pelis.to_csv("comedia_2010.csv", index=False, encoding='utf-8')
#df_pelis.to_csv("comedia_2011.csv", index=False, encoding='utf-8')
#df_pelis.to_csv("comedia_2012.csv", index=False, encoding='utf-8')
#df_pelis.to_csv("comedia_2013.csv", index=False, encoding='utf-8')
#df_pelis.to_csv("comedia_2014.csv", index=False, encoding='utf-8')
#df_pelis.to_csv("comedia_2015.csv", index=False, encoding='utf-8')
#df_pelis.to_csv("comedia_2016.csv", index=False, encoding='utf-8')
#df_pelis.to_csv("comedia_2017.csv", index=False, encoding='utf-8')
#df_pelis.to_csv("comedia_2018.csv", index=False, encoding='utf-8')
#df_pelis.to_csv("comedia_2019.csv", index=False, encoding='utf-8')
#df_pelis.to_csv("comedia_2020.csv", index=False, encoding='utf-8')
#df_pelis.to_csv("comedia_2021.csv", index=False, encoding='utf-8')
#df_pelis.to_csv("comedia_2022.csv", index=False, encoding='utf-8')
#df_pelis.to_csv("comedia_2023.csv", index=False, encoding='utf-8')
#df_pelis.to_csv("comedia_2024.csv", index=False, encoding='utf-8')

#### Cortometrajes Drama

In [616]:
#df_pelis.to_csv("drama_2010.csv", index=False, encoding='utf-8')
#df_pelis.to_csv("drama_2011.csv", index=False, encoding='utf-8')
#df_pelis.to_csv("drama_2012.csv", index=False, encoding='utf-8')
#df_pelis.to_csv("drama_2013.csv", index=False, encoding='utf-8')
#df_pelis.to_csv("drama_2014.csv", index=False, encoding='utf-8')
#df_pelis.to_csv("drama_2015.csv", index=False, encoding='utf-8')
#df_pelis.to_csv("drama_2016.csv", index=False, encoding='utf-8')
#df_pelis.to_csv("drama_2017.csv", index=False, encoding='utf-8')
#df_pelis.to_csv("drama_2018.csv", index=False, encoding='utf-8')
#df_pelis.to_csv("drama_2019.csv", index=False, encoding='utf-8')
#df_pelis.to_csv("drama_2020.csv", index=False, encoding='utf-8')
#df_pelis.to_csv("drama_2021.csv", index=False, encoding='utf-8')
#df_pelis.to_csv("drama_2022.csv", index=False, encoding='utf-8')
#df_pelis.to_csv("drama_2023.csv", index=False, encoding='utf-8')
#df_pelis.to_csv("drama_2024.csv", index=False, encoding='utf-8')